<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/rust01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ

1. Colab で実行しながら読まれることを想定している。
1. Rust の勉強
1. 資料は本家のテュートリアル https://rust-cli.github.io/book/index.html

# Rust をインストールする

In [1]:
# Rust をインストールする
%%capture
# !apt install rustc;

In [ ]:
%%writefile sh.rustup.sh
#!/bin/sh
# shellcheck shell=dash

# This is just a little script that can be downloaded from the internet to
# install rustup. It just does platform detection, downloads the installer
# and runs it.

# It runs on Unix shells like {a,ba,da,k,z}sh. It uses the common `local`
# extension. Note: Most shells limit `local` to 1 var per line, contra bash.

set -u

# If RUSTUP_UPDATE_ROOT is unset or empty, default it.
RUSTUP_UPDATE_ROOT="${RUSTUP_UPDATE_ROOT:-https://static.rust-lang.org/rustup}"

#XXX: If you change anything here, please make the same changes in setup_mode.rs
usage() {
    cat 1>&2 <<EOF
rustup-init 1.22.1 (76644d669 2020-07-08)
The installer for rustup

USAGE:
    rustup-init [FLAGS] [OPTIONS]

FLAGS:
    -v, --verbose           Enable verbose output
    -q, --quiet             Disable progress output
    -y                      Disable confirmation prompt.
        --no-modify-path    Don't configure the PATH environment variable
    -h, --help              Prints help information
    -V, --version           Prints version information

OPTIONS:
        --default-host <default-host>              Choose a default host triple
        --default-toolchain <default-toolchain>    Choose a default toolchain to install
        --default-toolchain none                   Do not install any toolchains
        --profile [minimal|default|complete]       Choose a profile
    -c, --component <components>...                Component name to also install
    -t, --target <targets>...                      Target name to also install
EOF
}

main() {
    downloader --check
    need_cmd uname
    need_cmd mktemp
    need_cmd chmod
    need_cmd mkdir
    need_cmd rm
    need_cmd rmdir

    get_architecture || return 1
    local _arch="$RETVAL"
    assert_nz "$_arch" "arch"

    local _ext=""
    case "$_arch" in
        *windows*)
            _ext=".exe"
            ;;
    esac

    local _url="${RUSTUP_UPDATE_ROOT}/dist/${_arch}/rustup-init${_ext}"

    local _dir
    _dir="$(mktemp -d 2>/dev/null || ensure mktemp -d -t rustup)"
    local _file="${_dir}/rustup-init${_ext}"

    local _ansi_escapes_are_valid=false
    if [ -t 2 ]; then
        if [ "${TERM+set}" = 'set' ]; then
            case "$TERM" in
                xterm*|rxvt*|urxvt*|linux*|vt*)
                    _ansi_escapes_are_valid=true
                ;;
            esac
        fi
    fi

    # check if we have to use /dev/tty to prompt the user
    local need_tty=yes
    for arg in "$@"; do
        case "$arg" in
            -h|--help)
                usage
                exit 0
                ;;
            -y)
                # user wants to skip the prompt -- we don't need /dev/tty
                need_tty=no
                ;;
            *)
                ;;
        esac
    done

    if $_ansi_escapes_are_valid; then
        printf "\33[1minfo:\33[0m downloading installer\n" 1>&2
    else
        printf '%s\n' 'info: downloading installer' 1>&2
    fi

    ensure mkdir -p "$_dir"
    ensure downloader "$_url" "$_file" "$_arch"
    ensure chmod u+x "$_file"
    if [ ! -x "$_file" ]; then
        printf '%s\n' "Cannot execute $_file (likely because of mounting /tmp as noexec)." 1>&2
        printf '%s\n' "Please copy the file to a location where you can execute binaries and run ./rustup-init${_ext}." 1>&2
        exit 1
    fi

    if [ "$need_tty" = "yes" ]; then
        # The installer is going to want to ask for confirmation by
        # reading stdin.  This script was piped into `sh` though and
        # doesn't have stdin to pass to its children. Instead we're going
        # to explicitly connect /dev/tty to the installer's stdin.
        if [ ! -t 1 ]; then
            err "Unable to run interactively. Run with -y to accept defaults, --help for additional options"
        fi

        # ignore "$_file" "$@" < /dev/tty
        ignore "$_file" "$@"
    else
        ignore "$_file" "$@"
    fi

    local _retval=$?

    ignore rm "$_file"
    ignore rmdir "$_dir"

    return "$_retval"
}

get_bitness() {
    need_cmd head
    # Architecture detection without dependencies beyond coreutils.
    # ELF files start out "\x7fELF", and the following byte is
    #   0x01 for 32-bit and
    #   0x02 for 64-bit.
    # The printf builtin on some shells like dash only supports octal
    # escape sequences, so we use those.
    local _current_exe_head
    _current_exe_head=$(head -c 5 /proc/self/exe )
    if [ "$_current_exe_head" = "$(printf '\177ELF\001')" ]; then
        echo 32
    elif [ "$_current_exe_head" = "$(printf '\177ELF\002')" ]; then
        echo 64
    else
        err "unknown platform bitness"
    fi
}

get_endianness() {
    local cputype=$1
    local suffix_eb=$2
    local suffix_el=$3

    # detect endianness without od/hexdump, like get_bitness() does.
    need_cmd head
    need_cmd tail

    local _current_exe_endianness
    _current_exe_endianness="$(head -c 6 /proc/self/exe | tail -c 1)"
    if [ "$_current_exe_endianness" = "$(printf '\001')" ]; then
        echo "${cputype}${suffix_el}"
    elif [ "$_current_exe_endianness" = "$(printf '\002')" ]; then
        echo "${cputype}${suffix_eb}"
    else
        err "unknown platform endianness"
    fi
}

get_architecture() {
    local _ostype _cputype _bitness _arch _clibtype
    _ostype="$(uname -s)"
    _cputype="$(uname -m)"
    _clibtype="gnu"

    if [ "$_ostype" = Linux ]; then
        if [ "$(uname -o)" = Android ]; then
            _ostype=Android
        fi
        if ldd --version 2>&1 | grep -q 'musl'; then
            _clibtype="musl"
        fi
    fi

    if [ "$_ostype" = Darwin ] && [ "$_cputype" = i386 ]; then
        # Darwin `uname -m` lies
        if sysctl hw.optional.x86_64 | grep -q ': 1'; then
            _cputype=x86_64
        fi
    fi

    case "$_ostype" in

        Android)
            _ostype=linux-android
            ;;

        Linux)
            _ostype=unknown-linux-$_clibtype
            _bitness=$(get_bitness)
            ;;

        FreeBSD)
            _ostype=unknown-freebsd
            ;;

        NetBSD)
            _ostype=unknown-netbsd
            ;;

        DragonFly)
            _ostype=unknown-dragonfly
            ;;

        Darwin)
            _ostype=apple-darwin
            ;;

        MINGW* | MSYS* | CYGWIN*)
            _ostype=pc-windows-gnu
            ;;

        *)
            err "unrecognized OS type: $_ostype"
            ;;

    esac

    case "$_cputype" in

        i386 | i486 | i686 | i786 | x86)
            _cputype=i686
            ;;

        xscale | arm)
            _cputype=arm
            if [ "$_ostype" = "linux-android" ]; then
                _ostype=linux-androideabi
            fi
            ;;

        armv6l)
            _cputype=arm
            if [ "$_ostype" = "linux-android" ]; then
                _ostype=linux-androideabi
            else
                _ostype="${_ostype}eabihf"
            fi
            ;;

        armv7l | armv8l)
            _cputype=armv7
            if [ "$_ostype" = "linux-android" ]; then
                _ostype=linux-androideabi
            else
                _ostype="${_ostype}eabihf"
            fi
            ;;

        aarch64)
            _cputype=aarch64
            ;;

        x86_64 | x86-64 | x64 | amd64)
            _cputype=x86_64
            ;;

        mips)
            _cputype=$(get_endianness mips '' el)
            ;;

        mips64)
            if [ "$_bitness" -eq 64 ]; then
                # only n64 ABI is supported for now
                _ostype="${_ostype}abi64"
                _cputype=$(get_endianness mips64 '' el)
            fi
            ;;

        ppc)
            _cputype=powerpc
            ;;

        ppc64)
            _cputype=powerpc64
            ;;

        ppc64le)
            _cputype=powerpc64le
            ;;

        s390x)
            _cputype=s390x
            ;;
        riscv64)
            _cputype=riscv64gc
            ;;
        *)
            err "unknown CPU type: $_cputype"

    esac

    # Detect 64-bit linux with 32-bit userland
    if [ "${_ostype}" = unknown-linux-gnu ] && [ "${_bitness}" -eq 32 ]; then
        case $_cputype in
            x86_64)
                _cputype=i686
                ;;
            mips64)
                _cputype=$(get_endianness mips '' el)
                ;;
            powerpc64)
                _cputype=powerpc
                ;;
            aarch64)
                _cputype=armv7
                if [ "$_ostype" = "linux-android" ]; then
                    _ostype=linux-androideabi
                else
                    _ostype="${_ostype}eabihf"
                fi
                ;;
            riscv64gc)
                err "riscv64 with 32-bit userland unsupported"
                ;;
        esac
    fi

    # Detect armv7 but without the CPU features Rust needs in that build,
    # and fall back to arm.
    # See https://github.com/rust-lang/rustup.rs/issues/587.
    if [ "$_ostype" = "unknown-linux-gnueabihf" ] && [ "$_cputype" = armv7 ]; then
        if ensure grep '^Features' /proc/cpuinfo | grep -q -v neon; then
            # At least one processor does not have NEON.
            _cputype=arm
        fi
    fi

    _arch="${_cputype}-${_ostype}"

    RETVAL="$_arch"
}

say() {
    printf 'rustup: %s\n' "$1"
}

err() {
    say "$1" >&2
    exit 1
}

need_cmd() {
    if ! check_cmd "$1"; then
        err "need '$1' (command not found)"
    fi
}

check_cmd() {
    command -v "$1" > /dev/null 2>&1
}

assert_nz() {
    if [ -z "$1" ]; then err "assert_nz $2"; fi
}

# Run a command that should never fail. If the command fails execution
# will immediately terminate with an error showing the failing
# command.
ensure() {
    if ! "$@"; then err "command failed: $*"; fi
}

# This is just for indicating that commands' results are being
# intentionally ignored. Usually, because it's being executed
# as part of error handling.
ignore() {
    "$@"
}

# This wraps curl or wget. Try curl first, if not installed,
# use wget instead.
downloader() {
    local _dld
    local _ciphersuites
    if check_cmd curl; then
        _dld=curl
    elif check_cmd wget; then
        _dld=wget
    else
        _dld='curl or wget' # to be used in error message of need_cmd
    fi

    if [ "$1" = --check ]; then
        need_cmd "$_dld"
    elif [ "$_dld" = curl ]; then
        get_ciphersuites_for_curl
        _ciphersuites="$RETVAL"
        if [ -n "$_ciphersuites" ]; then
            curl --proto '=https' --tlsv1.2 --ciphers "$_ciphersuites" --silent --show-error --fail --location "$1" --output "$2"
        else
            echo "Warning: Not enforcing strong cipher suites for TLS, this is potentially less secure"
            if ! check_help_for "$3" curl --proto --tlsv1.2; then
                echo "Warning: Not enforcing TLS v1.2, this is potentially less secure"
                curl --silent --show-error --fail --location "$1" --output "$2"
            else
                curl --proto '=https' --tlsv1.2 --silent --show-error --fail --location "$1" --output "$2"
            fi
        fi
    elif [ "$_dld" = wget ]; then
        get_ciphersuites_for_wget
        _ciphersuites="$RETVAL"
        if [ -n "$_ciphersuites" ]; then
            wget --https-only --secure-protocol=TLSv1_2 --ciphers "$_ciphersuites" "$1" -O "$2"
        else
            echo "Warning: Not enforcing strong cipher suites for TLS, this is potentially less secure"
            if ! check_help_for "$3" wget --https-only --secure-protocol; then
                echo "Warning: Not enforcing TLS v1.2, this is potentially less secure"
                wget "$1" -O "$2"
            else
                wget --https-only --secure-protocol=TLSv1_2 "$1" -O "$2"
            fi
        fi
    else
        err "Unknown downloader"   # should not reach here
    fi
}

check_help_for() {
    local _arch
    local _cmd
    local _arg
    _arch="$1"
    shift
    _cmd="$1"
    shift

    case "$_arch" in

        # If we're running on OS-X, older than 10.13, then we always
        # fail to find these options to force fallback
        *darwin*)
        if check_cmd sw_vers; then
            if [ "$(sw_vers -productVersion | cut -d. -f2)" -lt 13 ]; then
                # Older than 10.13
                echo "Warning: Detected OS X platform older than 10.13"
                return 1
            fi
        fi
        ;;

    esac

    for _arg in "$@"; do
        if ! "$_cmd" --help | grep -q -- "$_arg"; then
            return 1
        fi
    done

    true # not strictly needed
}

# Return cipher suite string specified by user, otherwise return strong TLS 1.2-1.3 cipher suites
# if support by local tools is detected. Detection currently supports these curl backends: 
# GnuTLS and OpenSSL (possibly also LibreSSL and BoringSSL). Return value can be empty.
get_ciphersuites_for_curl() {
    if [ -n "${RUSTUP_TLS_CIPHERSUITES-}" ]; then
        # user specified custom cipher suites, assume they know what they're doing
        RETVAL="$RUSTUP_TLS_CIPHERSUITES"
        return
    fi

    local _openssl_syntax="no"
    local _gnutls_syntax="no"
    local _backend_supported="yes"
    if curl -V | grep -q ' OpenSSL/'; then
        _openssl_syntax="yes"
    elif curl -V | grep -iq ' LibreSSL/'; then
        _openssl_syntax="yes"
    elif curl -V | grep -iq ' BoringSSL/'; then
        _openssl_syntax="yes"
    elif curl -V | grep -iq ' GnuTLS/'; then
        _gnutls_syntax="yes"
    else
        _backend_supported="no"
    fi

    local _args_supported="no"
    if [ "$_backend_supported" = "yes" ]; then
        # "unspecified" is for arch, allows for possibility old OS using macports, homebrew, etc.
        if check_help_for "notspecified" "curl" "--tlsv1.2" "--ciphers" "--proto"; then
            _args_supported="yes"
        fi
    fi

    local _cs=""
    if [ "$_args_supported" = "yes" ]; then
        if [ "$_openssl_syntax" = "yes" ]; then
            _cs=$(get_strong_ciphersuites_for "openssl")
        elif [ "$_gnutls_syntax" = "yes" ]; then
            _cs=$(get_strong_ciphersuites_for "gnutls")
        fi
    fi

    RETVAL="$_cs"
}

# Return cipher suite string specified by user, otherwise return strong TLS 1.2-1.3 cipher suites
# if support by local tools is detected. Detection currently supports these wget backends: 
# GnuTLS and OpenSSL (possibly also LibreSSL and BoringSSL). Return value can be empty.
get_ciphersuites_for_wget() {
    if [ -n "${RUSTUP_TLS_CIPHERSUITES-}" ]; then
        # user specified custom cipher suites, assume they know what they're doing
        RETVAL="$RUSTUP_TLS_CIPHERSUITES"
        return
    fi

    local _cs=""
    if wget -V | grep -q '\-DHAVE_LIBSSL'; then
        # "unspecified" is for arch, allows for possibility old OS using macports, homebrew, etc.
        if check_help_for "notspecified" "wget" "TLSv1_2" "--ciphers" "--https-only" "--secure-protocol"; then
            _cs=$(get_strong_ciphersuites_for "openssl")
        fi
    elif wget -V | grep -q '\-DHAVE_LIBGNUTLS'; then
        # "unspecified" is for arch, allows for possibility old OS using macports, homebrew, etc.
        if check_help_for "notspecified" "wget" "TLSv1_2" "--ciphers" "--https-only" "--secure-protocol"; then
            _cs=$(get_strong_ciphersuites_for "gnutls")
        fi
    fi

    RETVAL="$_cs"
}

# Return strong TLS 1.2-1.3 cipher suites in OpenSSL or GnuTLS syntax. TLS 1.2 
# excludes non-ECDHE and non-AEAD cipher suites. DHE is excluded due to bad 
# DH params often found on servers (see RFC 7919). Sequence matches or is
# similar to Firefox 68 ESR with weak cipher suites disabled via about:config.  
# $1 must be openssl or gnutls.
get_strong_ciphersuites_for() {
    if [ "$1" = "openssl" ]; then
        # OpenSSL is forgiving of unknown values, no problems with TLS 1.3 values on versions that don't support it yet.
        echo "TLS_AES_128_GCM_SHA256:TLS_CHACHA20_POLY1305_SHA256:TLS_AES_256_GCM_SHA384:ECDHE-ECDSA-AES128-GCM-SHA256:ECDHE-RSA-AES128-GCM-SHA256:ECDHE-ECDSA-CHACHA20-POLY1305:ECDHE-RSA-CHACHA20-POLY1305:ECDHE-ECDSA-AES256-GCM-SHA384:ECDHE-RSA-AES256-GCM-SHA384"
    elif [ "$1" = "gnutls" ]; then
        # GnuTLS isn't forgiving of unknown values, so this may require a GnuTLS version that supports TLS 1.3 even if wget doesn't.
        # Begin with SECURE128 (and higher) then remove/add to build cipher suites. Produces same 9 cipher suites as OpenSSL but in slightly different order.
        echo "SECURE128:-VERS-SSL3.0:-VERS-TLS1.0:-VERS-TLS1.1:-VERS-DTLS-ALL:-CIPHER-ALL:-MAC-ALL:-KX-ALL:+AEAD:+ECDHE-ECDSA:+ECDHE-RSA:+AES-128-GCM:+CHACHA20-POLY1305:+AES-256-GCM"
    fi 
}

main "$@" || exit 1

Overwriting sh.rustup.sh


In [4]:
!cd ~/

In [14]:
!cd ./share
!ls -la share

total 24
drwx------ 1 root root 4096 Nov 12 17:20 .
drwx------ 1 root root 4096 Nov 12 17:20 ..
drwx------ 1 root root 4096 Nov 12 17:21 jupyter


In [17]:
%env USER=root
!curl https://sh.rustup.rs -sSf | sh


env: USER=root
info: downloading installer
sh: 121: cannot open /dev/tty: No such device or address


In [ ]:
!source sh.rustup.sh

^C


In [ ]:
!apt remove rustc


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  gdb gdbserver libbabeltrace1 libc6-dbg libdw1 libhttp-parser2.7.1 libssh2-1
  libstd-rust-1.43 libstd-rust-dev rust-gdb
Use 'apt autoremove' to remove them.
The following packages will be REMOVED:
  cargo rustc
0 upgraded, 0 newly installed, 2 to remove and 12 not upgraded.
After this operation, 16.6 MB disk space will be freed.
(Reading database ... 145343 files and directories currently installed.)
Removing cargo (0.44.1-0ubuntu1~18.04.1) ...
Removing rustc (1.43.0+dfsg1+llvm-1~exp1ubuntu2~18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs 

In [ ]:
!ls -la /.cargo

ls: cannot access '/.cargo': No such file or directory


In [ ]:
# インストールの確認
!rustc --version

rustc 1.43.0


In [ ]:
# プログラムを書く
%%writefile main.rs
fn main() {
    println!("Hello, world!");
}

Overwriting main.rs


In [ ]:
# コンパイルして実行
!rustc main.rs
!./main

Hello, world!


In [ ]:
# 実験
# 名前を変えてみる
%%writefile hoge.rs
fn main() {
    println!("hoge, world!");
}

Overwriting hoge.rs


In [ ]:
# 実験
# 1行でコンパイルして、実行
!rustc hoge.rs; ./hoge

hoge, world!


# テュートリアルより



In [ ]:
# テュートリアルより最初の課題
# 次のような動作をするgrrsをつくる。

# $ cat test.txt
# foo: 10
# bar: 20
# baz: 30
# $ grrs foo test.txt
# foo: 10
# $ grrs --help
# [some help text explaining the available options]


In [ ]:
%%writefile grrs01.rs
fn main() {
    let pattern = std::env::args().nth(1).expect("no pattern given");
    let path = std::env::args().nth(2).expect("no path given");
}

Writing grrs01.rs


In [ ]:
# 実験
!rustc grrs01.rs
!./grrs01

In [ ]:
%env USER=root
!cargo init

env: USER=root
error: `cargo init` cannot be run on existing Cargo packages


In [ ]:
!cat Cargo.toml

[package]
name = "content"
version = "0.1.0"
authors = ["root"]
edition = "2018"

# See more keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html

[dependencies]
structopt = "0.3.13"

[[bin]]
name = "content"
path = "main.rs"

In [28]:
%%writefile Cargo.toml
[package]
name = "content"
version = "0.1.0"
authors = ["root"]
edition = "2018"

# See more keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html

[dependencies]
structopt = "0.3.13"

[[bin]]
name = "content"
path = "main.rs"

Writing Cargo.toml


In [ ]:
%%writefile main.rs
use structopt::StructOpt;

/// Search for a pattern in a file and display the lines that contain it.
#[derive(StructOpt)]
struct Cli {
    /// The pattern to look for
    pattern: String,
    /// The path to the file to read
    #[structopt(parse(from_os_str))]
    path: std::path::PathBuf,
}

fn main() {
    let pattern = std::env::args().nth(1).expect("no pattern given");
    let path = std::env::args().nth(2).expect("no path given");
}

Overwriting main.rs


In [ ]:
!rustc main.rs
!./main

error[E0432]: unresolved import `structopt`
 --> main.rs:1:5
  |
1 | use structopt::StructOpt;
  |     ^^^^^^^^^ maybe a missing crate `structopt`?

error: cannot determine resolution for the derive macro `StructOpt`
 --> main.rs:4:10
  |
4 | #[derive(StructOpt)]
  |          ^^^^^^^^^
  |
  = note: import resolution is stuck, try simplifying macro imports

error: cannot find attribute `structopt` in this scope
 --> main.rs:9:7
  |
9 |     #[structopt(parse(from_os_str))]
  |       ^^^^^^^^^

error: aborting due to 3 previous errors

For more information about this error, try `rustc --explain E0432`.
Hello, world!


In [ ]:
!ls -la


total 21284
drwxr-xr-x 1 root root    4096 Nov 13 11:09 .
drwxr-xr-x 1 root root    4096 Nov 13 01:43 ..
-rw-r--r-- 1 root root     258 Nov 13 11:11 Cargo.toml
drwxr-xr-x 1 root root    4096 Nov  6 17:30 .config
drwxr-xr-x 6 root root    4096 Nov 13 10:52 .git
-rw-r--r-- 1 root root     540 Nov 13 11:09 .gitignore
-rwxr-xr-x 1 root root 7252344 Nov 13 09:30 grrs01
-rw-r--r-- 1 root root     448 Nov 13 11:21 grrs01.rs
-rwxr-xr-x 1 root root 7248808 Nov 13 09:22 hoge
-rw-r--r-- 1 root root      43 Nov 13 09:21 hoge.rs
-rwxr-xr-x 1 root root 7248808 Nov 13 09:21 main
-rw-r--r-- 1 root root      44 Nov 13 09:21 main.rs
drwxr-xr-x 1 root root    4096 Nov  6 17:30 sample_data


In [24]:
# %%capture
!apt install rustc --fix-missing

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  cargo gdb gdbserver libbabeltrace1 libc6-dbg libdw1 libhttp-parser2.7.1
  libssh2-1 libstd-rust-1.43 libstd-rust-dev rust-gdb
Suggested packages:
  cargo-doc gdb-doc rust-doc rust-src lld-9
The following NEW packages will be installed:
  cargo gdb gdbserver libbabeltrace1 libc6-dbg libdw1 libhttp-parser2.7.1
  libssh2-1 libstd-rust-1.43 libstd-rust-dev rust-gdb rustc
0 upgraded, 12 newly installed, 0 to remove and 12 not upgraded.
Need to get 3,220 kB/87.3 MB of archives.
After this operation, 358 MB of additional disk space will be used.
Err:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 gdb amd64 8.1-0ubuntu3.2
  404  Not Found [IP: 91.189.88.152 80]
Err:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 gdbserver amd64 8.1-0ubuntu3.2
  404  Not Found [IP: 91.189.88.152 80]
Selecting previously unselected packag

In [25]:
!rustc --version

rustc 1.43.0


In [26]:
!cargo --version

cargo 1.43.0


In [35]:
!cargo install structopt

    Updating crates.io index
error: specified package `structopt v0.3.20` has no binaries


In [37]:
!cargo build


    Updating crates.io index
  Downloaded clap v2.33.3
  Downloaded structopt-derive v0.4.13
  Downloaded lazy_static v1.4.0
  Downloaded atty v0.2.14
  Downloaded bitflags v1.2.1
  Downloaded vec_map v0.8.2
  Downloaded textwrap v0.11.0
  Downloaded strsim v0.8.0
  Downloaded ansi_term v0.11.0
  Downloaded proc-macro2 v1.0.24
  Downloaded syn v1.0.48
  Downloaded quote v1.0.7
  Downloaded libc v0.2.80
  Downloaded proc-macro-error v1.0.4
  Downloaded heck v0.3.1
  Downloaded unicode-xid v0.2.1
  Downloaded version_check v0.9.2
  Downloaded unicode-segmentation v1.7.0
  Downloaded proc-macro-error-attr v1.0.4
  Downloaded unicode-width v0.1.8
   Compiling proc-macro2 v1.0.24
   Compiling version_check v0.9.2
   Compiling unicode-xid v0.2.1
   Compiling libc v0.2.80
   Compiling syn v1.0.48
   Compiling bitflags v1.2.1
   Compiling unicode-width v0.1.8
   Compiling unicode-segmentation v1.7.0
   Compiling ansi_term v0.11.0
   Compiling vec_map v0.8.2
   Compiling strsim v0.8.0
   Compil

!cargo build
したところ
error: couldn't read main.rs: No such file or directory (os error 2)

error: aborting due to previous error

error: could not compile `content`.
となった。
やはり main.rsである必要はあるみたい。
content
関係はまだわからない。